In [ ]:
import pandas as pd
import json

# Load the dataset
df = pd.read_csv("emergency_service_routing_with_timestamps.csv")

# Parse timestamp column if it's not already parsed
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Extract month-year for time trend analysis
df['MonthYear'] = df['Timestamp'].dt.to_period('M').astype(str)

# Chart 3: Incident trends over time
incident_trends = (
    df.groupby(['Region_Type', 'Incident_Type', 'MonthYear'])
    .size()
    .reset_index(name='Count')
    .to_dict(orient='records')
)

# create json file for incident trends
with open('incident_trends.json', 'w') as f3:
    json.dump(incident_trends, f3, indent=2)


In [2]:
import pandas as pd
import json

# Load the dataset
df = pd.read_csv("emergency_service_routing_with_timestamps.csv")

# Ensure timestamp column is parsed correctly
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Create a MonthYear column in 'YYYY-MM' format
df['MonthYear'] = df['Timestamp'].dt.to_period('M').astype(str)

# Group by Region_Type, Incident_Type, Incident_Severity, MonthYear
monthly_grouped = df.groupby(
    ['Region_Type', 'Incident_Type', 'Incident_Severity', 'MonthYear']
).size().reset_index(name='Count')

# Convert to list of dictionaries
monthly_data = monthly_grouped.to_dict(orient='records')

# Save to JSON
with open("severity_counts_monthly.json", "w") as f:
    json.dump(monthly_data, f, indent=2)

print("severity_counts_monthly.json has been saved successfully.")

severity_counts_monthly.json has been saved successfully.


In [ ]:
import pandas as pd
import json

# Load the CSV
df = pd.read_csv('emergency_service_routing_with_timestamps.csv')

# Convert timestamp to datetime and extract Month-Year
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['MonthYear'] = df['Timestamp'].dt.to_period('M').astype(str)

# Group by Region_Type, Incident_Type, MonthYear and count
monthly_counts = (
    df.groupby(['Region_Type', 'Incident_Type', 'MonthYear'])
    .size()
    .reset_index(name='Count')
)

# Convert to list of dicts
records = monthly_counts.to_dict(orient='records')

# Save to JSON file
with open('incident_type_counts_monthly.json', 'w') as f:
    json.dump(records, f, indent=2)

print("✅ incident_type_counts_monthly.json created successfully.")

✅ incident_type_counts_monthly.json created successfully.


In [ ]:
import pandas as pd
import numpy as np
import os

# File paths
input_csv = "emergency_service_routing_with_timestamps.csv"
output_dir = "public/data/"
os.makedirs(output_dir, exist_ok=True)

# Load dataset
df = pd.read_csv(input_csv)

# Convert Timestamp to datetime and extract Year
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df["MonthYear"] = df["Timestamp"].dt.to_period("M").astype(str)

# ----------------------------------------
# 1. Ambulance Response (Filtered)
# ----------------------------------------
ambulance_grouped = (
    df.groupby(["Ambulance_Availability", "Region_Type", "Emergency_Level", "MonthYear"])["Response_Time"]
    .mean()
    .reset_index()
)
ambulance_grouped.columns = [
    "Ambulance_Availability",
    "Region_Type",
    "Emergency_Level",
    "MonthYear",
    "Avg_Response_Time"
]
ambulance_output = os.path.join(output_dir, "ambulance_response_filtered.json")
ambulance_grouped.to_json(ambulance_output, orient="records", indent=2)

# ----------------------------------------
# 2. Injuries vs Response (with Region)
# ----------------------------------------
injury_grouped = (
    df.groupby(["Region_Type", "Emergency_Level", "Number_of_Injuries", "MonthYear"])["Response_Time"]
    .mean()
    .reset_index()
)
injury_grouped.columns = [
    "Region_Type",
    "Emergency_Level",
    "Number_of_Injuries",
    "MonthYear",
    "Avg_Response_Time"
]
injury_output = os.path.join(output_dir, "injuries_response.json")
injury_grouped.to_json(injury_output, orient="records", indent=2)

# ----------------------------------------
# 3. Heatmap: Distance Bin × Road Type
# ----------------------------------------
# Create distance bins
bins = [0, 10, 20, 30, 40, 50, np.inf]
labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50+']
df["Distance_Bin"] = pd.cut(df["Distance_to_Incident"], bins=bins, labels=labels, right=False)

# Group by all filters + visual dimensions
heatmap_grouped = (
    df.groupby(["Road_Type", "Distance_Bin", "Region_Type", "Emergency_Level", "MonthYear"])["Response_Time"]
    .mean()
    .reset_index()
    .dropna()
)

heatmap_grouped.columns = [
    "Road_Type", 
    "Distance_Bin", 
    "Region_Type", 
    "Emergency_Level",
    "MonthYear",
    "Avg_Response_Time"
]

heatmap_output = os.path.join(output_dir, "response_heatmap.json")
heatmap_grouped.to_json(heatmap_output, orient="records", indent=2)

print("All JSON files generated in:", output_dir)

All JSON files generated in: public/data/


/var/folders/rw/5klndzr95nvf78n2v4xxbfpw0000gn/T/ipykernel_3911/2075533417.py:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["Road_Type", "Distance_Bin", "Region_Type", "Emergency_Level", "MonthYear"])["Response_Time"]


In [ ]:
import pandas as pd
import os

# Input & output setup
input_csv = "emergency_service_routing_with_timestamps.csv"
output_dir = "public/data/"
os.makedirs(output_dir, exist_ok=True)

# Load data
df = pd.read_csv(input_csv)

# Parse timestamp and extract month-year
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df["MonthYear"] = df["Timestamp"].dt.to_period("M").astype(str)

# Drop rows with missing values in key fields
df = df.dropna(subset=[
    "Weather_Condition",
    "Road_Type",
    "Traffic_Congestion",
    "Region_Type",
    "Response_Time"
])

# Group by relevant dimensions
weather_heatmap = (
    df.groupby([
        "Weather_Condition",       # Y-axis
        "Road_Type",               # X-axis
        "Traffic_Congestion",      # Global filter
        "Region_Type",             # Global filter
        "MonthYear"                # Global filter
    ])["Response_Time"]
    .mean()
    .reset_index()
)

# Rename column
weather_heatmap.rename(columns={"Response_Time": "Avg_Response_Time"}, inplace=True)

# Export to JSON
output_path = os.path.join(output_dir, "weather_heatmap.json")
weather_heatmap.to_json(output_path, orient="records", indent=2)

print("✅ weather_heatmap.json generated at:", output_path)

In [7]:
import pandas as pd

# Load the full dataset
df = pd.read_csv("emergency_service_routing_with_timestamps.csv")

# Sample random rows
sampled_df = df.sample(n=12000)

sampled_df.reset_index(drop=True, inplace=True)

# Save the subset for testing
sampled_df.to_csv("sampled_emergency_data_3.csv", index=False)

print("Sampled data saved to sampled_emergency_data_3.csv")

Sampled data saved to sampled_emergency_data_3.csv
